#### <span style="color:blue">*JUST RUN CELL*</span>

In [1]:
import os 
os.chdir(os.path.join('/home/datawork-osmose',[s for s in os.getcwd().split('/') if 'OSmOSEanalytics' in s][0],'source'))
from launcher_spectrogramGenerator import main, job_monitoring, visu_adjustSpectros, params_recommendation, display_metadata, list_datasets, params_checkingNberPixels

## <span style="color:red">*FILL & RUN CELLS*</span> Dataset preparation

- ``dataset_ID`` is the name of the dataset to be processed;
- ``analysis_fs`` is the sample frequency you want to use for your analysis, which can be different from the original one.

In [2]:
list_datasets()

Available datasets:
  - APOCADO_IROISE_C2D1_07072022
  - APOCADO_test_IROISE_d1
  - APOCADO_test_PASSEPARTOUT
  - argoLOV
  - boussole_MERMAID
  - CETIROISE_POINT_E_20220725
  - CETIROISE_POINT_G_20220509
  - CETIROISE_POINT_G_PHASE_1
  - CETIROISE_PONIT_B_20220717
  - DDU2018
  - fecampOWFSOMM
  - fromveur256000
  - fromveur256000_SelectionPourAnnotations
  - gliderSPAms
  - gliderSPAms_PAMGuard
  - gliderSPAms_sample1
  - gliderWalterShoals
  - MAHY
  - MAYOBS
  - MAYOBS_MOAF
  - MAYOBS_MOCA
  - MPSU_ForestouHuella
  - ohasisbio2015wker
  - ohasisbio2015wker_sample2
  - SPMAuralA_2011
  - SPMAuralA_2011_SelectionPourAnnotations2
  - SPMAuralA_2011_SelectionPourAnnotations2_copy
  - TP_annotation_HF_CETIROISE


In [3]:
dataset_ID = 'boussole_MERMAID'
display_metadata(dataset_ID)

Original sample frequency (Hz) : 50000
2021-10-11T10:22  -->  2022-03-08T16:38
Cumulated number of days : 148
Original audio file duration (s) : 1181
Total number of files: 8645
Total volume (GB): 1532.3
Auxiliary files : []
***************************
Existing analysis paramaters (fileDuration_sampleFrequency) : ['30_50000']


In [4]:
analysis_fs = 2000

### Dataset subset

Note that you can process only a subset of your entire dataset by creating the file `/home/datawork-osmose/dataset/dataset_ID/analysis/subset_files.csv`, which is a simple list of files to be processed, for example:

`% head /home/datawork-osmose/dataset/fecampOWFSOMM/analysis/subset_files.csv
channelA_2020_11_20_15_40_17.wav
channelA_2020_11_20_15_43_20.wav
channelA_2020_11_20_16_20_17.wav
channelA_2020_11_20_16_23_20.wav
channelA_2020_11_20_16_30_17.wav
channelA_2020_11_20_16_33_20.wav
channelA_2020_11_20_16_43_20.wav
channelA_2020_11_20_16_50_17.wav
channelA_2020_11_20_16_53_20.wav
channelA_2020_11_20_17_10_17.wav
`

## <span style="color:red">*FILL & RUN CELLS*</span> Configure spectrogram parameters

### Main parameters 

Start by setting the value of `maxtime_display_spectro` in seconds. It corresponds to the maximal duration of the spectrogram display window. It must be inferior or equal to your original audio file duration. You also have to specify how you want to proceed with file segmentation : `pad_silence = 1` will preserve your spectrogram duration by padding with silence the last remaining segment, otherwise set `pad_silence = 0`.

Then, you can set the value of `nber_tile_level`, which is the number of zoom levels you want (they are used in our web-based annotation tool APLOSE). With `nber_tile_level = 1`, your shortest spectrogram display window has a duration of `maxtime_display_spectro` seconds (that is no zoom at all) ; with `nber_tile_level = 2`, a duration of `maxtime_display_spectro`/2 seconds ; with `nber_tile_level = 3`, a duration of `maxtime_display_spectro`/4 seconds ...

After that, you can set the following classical spectrogram parameters : `nfft` (in samples), `winsize` (in samples), `overlap` (in \%). **Note that with those parameters you set the resolution of your spectrogram display window with the smallest duration, obtained with the highest zoom level.**

In [23]:
maxtime_display_spectro = 60 # this default value corresponds to your audio file duration
pad_silence = 1

nber_tile_level = 4

nfft = 1024 # samples
winsize = 1024# samples
overlap = 99  # %

### Amplitude normalization 

Eventually, we also propose you different modes of data/spectrogram normalization.

Normalization over raw data samples with the variable `data_normalization` (default value `''`, i.e. no normalization) :
- instrument-based normalization with the three parameters `sensitivity_dB` (in dB, default value = 0), `gain` (in dB, default value = 0) and `peak_voltage` (in V, default value = 1). Using default values, no normalization will be performed ;

- z-score normalization over a given time period through the variable `zscore_duration`, applied directly on your raw timeseries. The possible values are:
    - `zscore_duration = 'original'` : the audio file duration will be used as time period ;
    - `zscore_duration = '10H'` : any time period put as a string using classical [time alias](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#timeseries-offset-aliases). This period should be higher than your file duration. 

Normalization over spectra with the variable `spectro_normalization` (default value `'density'`, see OSmOSEanalytics/documentation/theory_spectrogram.pdf for details) :
- density-based normalization by setting `spectro_normalization = 'density'`
- spectrum-based normalization by setting `spectro_normalization = 'spectrum'` 

In the cell below, you can also have access to the amplitude dynamics in dB throuh the parameters `max_color_val` and `min_color_val`, the colormap `colmapspectros` to be used (see possible options in the [documentation](https://matplotlib.org/stable/tutorials/colors/colormaps.html)) and specify the frequency cut `fmin_HighPassFilter` of a high-pass filter if needed.

In [30]:
data_normalization = 'instrument' # 'instrument' OR 'zscore'

zscore_duration = 'original' # parameter for 'zscore' mode, values = time alias OR 'original' 
sensitivity_dB = -164 # parameters for 'instrument' mode
gain_dB = 14.7 # parameter for 'instrument' mode
peak_voltage = 2.5 # parameter for 'instrument' mode

spectro_normalization = 'density' # 'density' OR 'spectrum' 


sensitivity_dB = -170
gain_dB = 10
peak_voltage = 1.5

max_color_val = 140
min_color_val = 0

fmin_HighPassFilter = 10
colmapspectros = 'viridis'

### Parameter adjustement 


In the cell below you can **check your spectrogram dimension w.r.t your screen resolution** (just run it). We calculate the number of time windows (or equivalently, the number of spectra) you have in your shortest spectrogram display window.

Be aware that this number should be as close as your horizontal screen resolution (ie approximately 2000 pixels, as a classical screen resolution is 1920x1080 pixels (horizontal pixels) x (vertical pixels) ) to avoid numerical compression during image display on your screen, as well as useless over-resoluted spectrograms obtained at a high computational cost. We warn you if you are higher, but you can still compute higher-resolution spectrograms if you want.

In [31]:
params_checkingNberPixels(analysis_fs,maxtime_display_spectro,nfft,winsize,overlap,nber_tile_level,dataset_ID)

your smallest tile has a duration of: 7.5 (s)




your number of time windows in this tile is: 1270


your resolutions : time =  0.006 (s) / frequency =  1.953 (Hz)


`nberAdjustSpectros` is the number of spectrogram examples used to adjust your parameters. If you are really not sure about your parameters, it is better to start with a small number, because each time you will have to wait for the generation of all your `nberAdjustSpectros` (x the different zoom levels) spectrograms before being able to re-generate spectrograms with another set of parameters.

In [32]:
nberAdjustSpectros = 2

## <span style="color:blue">*JUST RUN CELL*</span> Adjust spectrogram parameters

### Compute `nberAdjustSpectros` spectrograms to adjust parameters. 

In [33]:
main(dataset_ID,analysis_fs, nfft ,winsize,nberAdjustSpectros,overlap,colmapspectros,nber_tile_level,min_color_val,max_color_val,maxtime_display_spectro,data_normalization,fmin_HighPassFilter,sensitivity_dB,peak_voltage,spectro_normalization,gain_dB,False,pad_silence,zscore_duration)

Generating 2 spectrograms (x the number of zoom levels) for parameter adjustment! In a few seconds, your first spectrograms will be visible in the next cell..


### Visualize `nberAdjustSpectros` spectrograms to adjust parameters. 

Re-run several times this cell to update the folder of images because they keep being generated while you visualize them. If this set of parameters does not suit you, change them and re-run new spectrograms with the previous cells, as many times as you want.

In [35]:
visu_adjustSpectros()

Spectrograms not ready yet, please wait a bit..


## <span style="color:blue">*JUST RUN CELL*</span>  Launch spectrogram generation

Just one thing : if you create your spectrograms for an APLOSE campaign, set `write_datasets_csv_for_APLOSE=True` below !

In [14]:
write_datasets_csv_for_APLOSE=False
main(dataset_ID,analysis_fs, nfft ,winsize,0,overlap,colmapspectros,nber_tile_level,min_color_val,max_color_val,maxtime_display_spectro,data_normalization,fmin_HighPassFilter,sensitivity_dB,peak_voltage,spectro_normalization,gain_dB,write_datasets_csv_for_APLOSE,pad_silence,zscore_duration)

OK we are generating all your spectrograms now ! 
 
 You can look at the progress bar in the cell below, or you can leave Jupyter and come back to Earth, your job is done, we will mail you when it is done ;) 


### Track progress

In [ ]:
job_monitoring(dataset_ID,analysis_fs,nfft,winsize,overlap,min_color_val,max_color_val)